In [97]:
import pandas as pd
import numpy as np
import re

In [109]:
Newlogs = pd.read_csv('data/17.04.csv') #need to change manually!!!!!
AllData = pd.read_csv('data/allDiscussionsData.csv')
UsersWithId = pd.read_csv('data/UsersWithId.csv')
last_update = pd.read_csv('data/lastUpdate_Discussions.csv')
inquisitive = 5
Curios = 20
Socrates  = 40
weekNo = 'week7'#need to change manually!!!!!

In [111]:
Newlogs['datetime']= pd.to_datetime(Newlogs['Time'], format="%d/%m/%y, %H:%M")
last_date = Newlogs.datetime.max()
# last_date = pd.Series(last_date, name = 'TimeStamp')
# last_date = pd.DataFrame(last_date)
# last_date.to_csv('data/lastUpdate_Discussions.csv',index=False)


In [112]:
last_update = pd.read_csv('data/lastUpdate_Discussions.csv')

In [119]:
pd.to_datetime(last_update.iloc[0].TimeStamp) < last_date

True

Timestamp('2020-04-17 11:53:00')

In [92]:
Newlogs.datetime.max()

Timestamp('2020-04-17 11:53:00')

In [81]:
def DeletedDiscussions(data): 
    DeletedDiscussions = pd.DataFrame()
    for index, row in data.iterrows():
        if  (row['Component'] =='Discussion deleted'):
            DiscussinId = re.search('has deleted the discussion with id (.+?) in the forum', row['Event name'])
            if DiscussinId:
                found = DiscussinId.group(1)
                DeletedDiscussions = DeletedDiscussions.append({'DiscussinId': found},ignore_index=True)
    return DeletedDiscussions

In [82]:
DeletedDiscussions = DeletedDiscussions(Newlogs)


In [83]:
DeletedDiscussions.to_csv('data/DeletedPosts.csv')

In [84]:
def IsDeleted(row):
    DiscussinId = re.search('has created the discussion with id (.+?) in the forum', row['Event name'])
    if DiscussinId:
        found = DiscussinId.group(1)
        return  pd.Series(DeletedDiscussions['DiscussinId']).str.contains(found, regex=False).any()
             
                    

In [85]:
def calculateOutput(data) :# adding 1 if the row is a Discussion 
    last_update_Date = pd.to_datetime(last_update['last_Date'].iloc[0])
    for index, row in data.iterrows():
        name =  row['User full name']
        rowTime = pd.to_datetime(row['Time'].replace(',', ''))
#         if  np.logical_and(row['Component'] =='Discussion created' , rowTime > last_update_Date):
        if  (row['Component'] =='Discussion created'):
            if (not IsDeleted(row)):
                data.loc[index,weekNo] = 1  
            else : 
                data.loc[index,weekNo] = 0
        else :
            data.loc[index,weekNo] = 0 
    return data.reset_index()

In [86]:
UpdateData = calculateOutput(Newlogs)


KeyError: 'User full name'

In [9]:
def Summarizer(UpdateData) :
    sume = np.sum(UpdateData[weekNo])
    s2 = pd.Series(sume, name = weekNo)
    
    return pd.concat([s2], axis=1)

In [10]:
UpdateData = UpdateData.groupby('User full name').apply(lambda x: Summarizer(x)).reset_index()

In [11]:
UpdateData = UpdateData[UpdateData[weekNo] >0 ]

In [12]:
UpdateData

,User full name,level_1,week6
0,Kobi Gal,0,2.0
8,אורי זלוטקין,0,1.0
9,אורי סלומון,0,1.0
15,איתי אסרף,0,2.0
20,אלון מלאך,0,1.0
22,אלון שניידר,0,1.0
26,אמיר סולומון זהבי,0,1.0
31,גיא אלוביץ,0,1.0
38,דור לוי,0,1.0
39,דור שרטו,0,1.0


In [13]:
UpdateData = UpdateData.drop(['level_1'], axis=1)

In [14]:
result = pd.merge(AllData, UpdateData, how='outer', on=['User full name'])

In [15]:
result

,Sum,inquisitive,Curios,Socrates,User full name,week6
0,NaN,NaN,NaN,NaN,Kobi Gal,2.0
1,NaN,NaN,NaN,NaN,אורי זלוטקין,1.0
2,NaN,NaN,NaN,NaN,אורי סלומון,1.0
3,NaN,NaN,NaN,NaN,איתי אסרף,2.0
4,NaN,NaN,NaN,NaN,אלון מלאך,1.0
5,NaN,NaN,NaN,NaN,אלון שניידר,1.0
6,NaN,NaN,NaN,NaN,אמיר סולומון זהבי,1.0
7,NaN,NaN,NaN,NaN,גיא אלוביץ,1.0
8,NaN,NaN,NaN,NaN,דור לוי,1.0
9,NaN,NaN,NaN,NaN,דור שרטו,1.0


In [16]:
result['Sum']= result.drop(['inquisitive','Curios','Socrates','Sum'], axis=1).sum(axis=1) #summing all weeks

In [17]:
result

,Sum,inquisitive,Curios,Socrates,User full name,week6
0,2.0,NaN,NaN,NaN,Kobi Gal,2.0
1,1.0,NaN,NaN,NaN,אורי זלוטקין,1.0
2,1.0,NaN,NaN,NaN,אורי סלומון,1.0
3,2.0,NaN,NaN,NaN,איתי אסרף,2.0
4,1.0,NaN,NaN,NaN,אלון מלאך,1.0
5,1.0,NaN,NaN,NaN,אלון שניידר,1.0
6,1.0,NaN,NaN,NaN,אמיר סולומון זהבי,1.0
7,1.0,NaN,NaN,NaN,גיא אלוביץ,1.0
8,1.0,NaN,NaN,NaN,דור לוי,1.0
9,1.0,NaN,NaN,NaN,דור שרטו,1.0


In [18]:
def IssueBadges(result):
    IssueToUsers = pd.DataFrame()
    for index, row in result.iterrows():
        B1 = 0
        B2 = 0
        B3 = 0
        if  (np.logical_and(row['Sum'] >= inquisitive  , row.inquisitive != 1)): 
            result.loc[index,'inquisitive'] = 1 
            B1=1
        if  np.logical_and(row['Sum'] >= Curios , row.Curios != 1): 
            result.loc[index,'Curios'] = 1 
            B2=1
        if  np.logical_and(row['Sum'] >= Socrates , row.Socrates != 1): 
            result.loc[index,'Socrates'] = 1 
            B3=1
        IssueToUsers =  IssueToUsers.append({'User full name': row['User full name'],'inquisitive' :B1,'Curios' :B2,'Socrates' :B3}, ignore_index=True)
    return IssueToUsers

In [19]:
AllData = result 

In [20]:
ToIssueToday = IssueBadges(result)

In [21]:
ToIssueToday

,Curios,Socrates,User full name,inquisitive
0,0.0,0.0,Kobi Gal,0.0
1,0.0,0.0,אורי זלוטקין,0.0
2,0.0,0.0,אורי סלומון,0.0
3,0.0,0.0,איתי אסרף,0.0
4,0.0,0.0,אלון מלאך,0.0
5,0.0,0.0,אלון שניידר,0.0
6,0.0,0.0,אמיר סולומון זהבי,0.0
7,0.0,0.0,גיא אלוביץ,0.0
8,0.0,0.0,דור לוי,0.0
9,0.0,0.0,דור שרטו,0.0


In [22]:
UsersWithId = UsersWithId.rename(columns={UsersWithId.columns[0]: "User full name",UsersWithId.columns[1]: "id"})

In [23]:
ToIssueToday = pd.merge(ToIssueToday, UsersWithId, how='left', on=['User full name'])

In [24]:
ToIssueToday =  ToIssueToday[ToIssueToday['id'].notna()]


In [25]:
ToIssueToday

,Curios,Socrates,User full name,inquisitive,id
1,0.0,0.0,אורי זלוטקין,0.0,302390778.0
2,0.0,0.0,אורי סלומון,0.0,316306323.0
3,0.0,0.0,איתי אסרף,0.0,322601907.0
4,0.0,0.0,אלון מלאך,0.0,207024878.0
5,0.0,0.0,אלון שניידר,0.0,312213978.0
6,0.0,0.0,אמיר סולומון זהבי,0.0,206655839.0
7,0.0,0.0,גיא אלוביץ,0.0,208726935.0
8,0.0,0.0,דור לוי,0.0,316614726.0
9,0.0,0.0,דור לוי,0.0,204459952.0
10,0.0,0.0,דור שרטו,0.0,312204811.0


In [93]:
#updating the Last update date :
last_date = Newlogs.datetime.max()# converting str to timeStamp and takes the last date
last_date = pd.DataFrame(last_date)
last_date.to_csv('data/lastUpdate_Discussions.csv',index=False)
result.to_csv('data/allDiscussionsData.csv',index=False)
ToIssueToday.to_csv('data/ToIssueToday_Discussions.csv',index=False)